In [2]:
from classes.ArquivoInspecao import ArquivoInspecao

try:
    arquivo = ArquivoInspecao("README.md")
    if arquivo.verifica:
        print("Arquivo CSV inspecionado com sucesso")
    else:
        print("O arquivo possui dados ausentes ou não possui dados")
except Exception as e:
    print(f"Erro ao inspecionar o arquivo: {e}")

Erro ao inspecionar o arquivo: O arquivo não é um CSV!
